![QuantConnect Logo](https://cdn.quantconnect.com/web/i/icon.png)
<hr>

# Strategy 1: Buy and hold

Buy and hold generates profit from the risk premia of market risk exposition.

A buy and hold strategy of 1 instrument is the most simple strategy. It requires several considerations that will be touched on in this notebook

## Data aquisition

In [24]:
from datetime import datetime
from QuantConnect import Resolution
from QuantConnect.Securities import Futures
from helper import cumulated_returns, calculate_return, calculate_percentage_return, calculate_return_in_usd, calculate_return_in_mxn, calculate_transaction_costs, expiration_dates


In [25]:
# Initialize QuantBook
qb = QuantBook()

# Add Future Subscriptions for ES and MES with Daily resolution
es_future = qb.AddFuture(Futures.Indices.SP500EMini, Resolution.Daily)
mes_future = qb.AddFuture(Futures.Indices.MicroSP500EMini, Resolution.Daily)

# Add Forex Subscription for USD/MXN with Daily resolution
fx_pair = qb.AddForex("USDMXN").symbol


# Set Historical Data Range
start_date = datetime(2007, 1, 1)
end_date = datetime.now()

# Request Historical Data for ES and MES futures
es_history = qb.History([es_future.Symbol], start_date, end_date, Resolution.Daily)
mes_history = qb.History([mes_future.Symbol], start_date, end_date, Resolution.Daily)

# Request Historical Data for USD/MXN FX pair
fx_history = qb.History([fx_pair], start_date, end_date, Resolution.Daily)

# Save data to Object Store (optional)
# qb.ObjectStore.Save("ES_Future_Data", es_history.to_csv(index=False))
# qb.ObjectStore.Save("MES_Future_Data", mes_history.to_csv(index=False))

es_history.reset_index(level=0, drop=True, inplace=True)
mes_history.reset_index(level=0, drop=True, inplace=True)
# Reset index for the historical data

# Display the first few rows of the historical data
print("ES Future Data:")
print(es_history.head())

print("\nMES Future Data:")
print(mes_history.head())

# Display the first few rows of the historical data
print("\nUSD/MXN FX Data:")
print(fx_history.head())


ES Future Data:
                               askclose      askhigh       asklow  \
symbol time                                                         
ES 1S1 2007-01-03 19:00:00  1380.443221  1394.484909  1371.969788   
       2007-01-04 19:00:00  1383.348398  1386.253575  1372.211887   
       2007-01-05 19:00:00  1371.727690  1383.348398  1369.790906   
       2007-01-08 19:00:00  1378.022240  1379.474829  1368.580415   
       2007-01-09 19:00:00  1376.327554  1379.959025  1369.548808   

                                askopen  asksize     bidclose      bidhigh  \
symbol time                                                                  
ES 1S1 2007-01-03 19:00:00  1388.674556    173.0  1380.201123  1394.242811   
       2007-01-04 19:00:00  1380.443221    510.0  1383.106300  1386.011477   
       2007-01-05 19:00:00  1383.348398     30.0  1371.243494  1383.106300   
       2007-01-08 19:00:00  1371.727690    103.0  1377.780142  1379.232731   
       2007-01-09 19:00:00  1378

In [26]:
print(es_history.index)
print(mes_history.index)
print(fx_history.index)

MultiIndex([('ES 1S1', '2007-01-03 19:00:00'),
            ('ES 1S1', '2007-01-04 19:00:00'),
            ('ES 1S1', '2007-01-05 19:00:00'),
            ('ES 1S1', '2007-01-08 19:00:00'),
            ('ES 1S1', '2007-01-09 19:00:00'),
            ('ES 1S1', '2007-01-10 19:00:00'),
            ('ES 1S1', '2007-01-11 19:00:00'),
            ('ES 1S1', '2007-01-12 19:00:00'),
            ('ES 1S1', '2007-01-16 19:00:00'),
            ('ES 1S1', '2007-01-17 19:00:00'),
            ...
            ('ES 1S1', '2024-05-06 20:00:00'),
            ('ES 1S1', '2024-05-07 20:00:00'),
            ('ES 1S1', '2024-05-08 20:00:00'),
            ('ES 1S1', '2024-05-09 20:00:00'),
            ('ES 1S1', '2024-05-10 20:00:00'),
            ('ES 1S1', '2024-05-13 20:00:00'),
            ('ES 1S1', '2024-05-14 20:00:00'),
            ('ES 1S1', '2024-05-15 20:00:00'),
            ('ES 1S1', '2024-05-16 20:00:00'),
            ('ES 1S1', '2024-05-17 20:00:00')],
           names=['symbol', 'time'], length

## Instrument dictionary

Each contract may have different multipliers. a multiplier of 50 means that a 1$ move is equal to a 50$ change in returns for each contract one is exposed to. We will create a dictionary with the relevant information to calculate the returns down the road.

### Compute Functions

In [27]:
instruments = {
    'ES': {
        'multiplier': 50,
        'tick_value': 0.25,
        'minimum_fluctuation': 12.50,
        'spread': 0.25,  # Typical spread
        'commission': 2.50,  # Typical commission per contract
        'expiration_dates': expiration_dates['ES']
    },
    'MES': {
        'multiplier': 5,
        'tick_value': 0.25,
        'minimum_fluctuation': 1.25,
        'spread': 0.25,  # Typical spread
        'commission': 2.50,  # Typical commission per contract
        'expiration_dates': expiration_dates['MES']
    }
}


## Summary statistics
We can begin working with the aquired data to calculate some useful statistics, such as returns, standard deviations, skew and fat tail ratios. Use the helper function to process these statistics.

### Returns

In [28]:
# Definir una función para calcular y unir retornos para un símbolo dado
def calculate_and_merge_returns(symbol, price_series, instruments):
    """
    Inputs 
    symbol: str - a symbol to calculate returns for
    price_series: pd.Series - a price series to calculate returns from
    instruments: dict - a dictionary of instruments with metadata

    Returns
    merged_df: pd.DataFrame - a DataFrame with both return and percentage return columns
    """
    returns_df = calculate_return(symbol, price_series, instruments)
    percentage_returns = calculate_percentage_return(symbol, price_series, instruments, num_contracts=1)
    
    # Unir los DataFrames en base al índice
    merged_df = returns_df.join(percentage_returns.rename('percentage_return'))
    return merged_df


In [29]:

# Inicializar un diccionario para almacenar los resultados
merged_returns = {}

# Iterar sobre cada símbolo en el diccionario de instrumentos
for symbol in instruments.keys():
    if symbol == 'ES':
        price_series = es_history['close']
    elif symbol == 'MES':
        price_series = mes_history['close']
    
    # Calcular y unir los retornos
    merged_returns[symbol] = calculate_and_merge_returns(symbol, price_series, instruments)
    
# Mostrar los primeros registros de los DataFrames unidos
for symbol, df in merged_returns.items():
    print(f"{symbol} Merged Returns Data:")
    print(df.head())



ES Merged Returns Data:
                            price_return  currency_return  percentage_return
symbol time                                                                 
ES 1S1 2007-01-04 19:00:00      2.905177       145.258845           0.210489
       2007-01-05 19:00:00    -11.378610      -568.930475          -0.822685
       2007-01-08 19:00:00      6.294550       314.727497           0.458878
       2007-01-09 19:00:00     -1.694687       -84.734326          -0.122980
       2007-01-10 19:00:00      4.841961       242.098075           0.351803
MES Merged Returns Data:
                             price_return  currency_return  percentage_return
symbol  time                                                                 
MES 1S1 2019-05-07 20:00:00    -19.875188       -99.375939          -0.652921
        2019-05-08 20:00:00     -6.799406       -33.997032          -0.224836
        2019-05-09 20:00:00    -13.075781       -65.378907          -0.433351
        2019-05-10 20:

## Backtest buy and hold 1 contract. Return is a dataframe or time series.

In [37]:

# Definir la función de simulación
def simulate_buy_and_hold(history_df, symbol, fx_rates_df, num_contracts=1):
    params = instruments[symbol]
    initial_price = history_df['close'].iloc[0]
    final_price = history_df['close'].iloc[-1]

    total_return_in_usd = calculate_return_in_usd(symbol, initial_price, final_price, instruments)
    total_return_in_mxn = calculate_return_in_mxn(total_return_in_usd, fx_rates_df)

    # Calcular costos de transacción
    transaction_costs = calculate_transaction_costs(params, history_df, start_date, end_date)

    # Ajustar retornos por costos de transacción
    net_return_in_usd = total_return_in_usd - transaction_costs
    net_return_in_mxn = calculate_return_in_mxn(net_return_in_usd, fx_rates_df)

    # Multiplicar los retornos por el número de contratos
    net_return_in_usd *= num_contracts
    net_return_in_mxn *= num_contracts

    return {
        "return_in_points": (final_price - initial_price) * num_contracts,
        "return_in_usd": net_return_in_usd,
        "return_in_mxn": net_return_in_mxn
    }

In [44]:

# Llamar a la función y asignar los valores retornados a variables
num_contracts = 1  # Ejemplo: se pueden ajustar los contratos según sea necesario
result = simulate_buy_and_hold(es_history, symbol, fx_history, num_contracts)

# Asignar los valores retornados a variables específicas
return_in_points = result["return_in_points"]
return_in_usd = result["return_in_usd"]
return_in_mxn = result["return_in_mxn"]

# Imprimir fechas
print(f"Start Date: {es_history.index[0]}")
print(f"End Date: {es_history.index[-1]}")
# Imprimir los resultados
print(f"Return in Points: {return_in_points}")
print(f"Return in USD: {return_in_usd}")
print(f"Return in MXN: {return_in_mxn}")


Start date type: <class 'datetime.datetime'>
End date type: <class 'datetime.datetime'>
Start Date: ('ES 1S1', Timestamp('2007-01-03 19:00:00'))
End Date: ('ES 1S1', Timestamp('2024-05-17 20:00:00'))
Return in Points: 3947.5488770265238
Return in USD: 19587.744385132617
Return in MXN: 313099.0039878057
